In [1]:
import pandas as pd
from datetime import date
from google.colab import drive

drive.mount('/content/drive')

print("--- Starting Comprehensive Data Preparation for Tableau ---")


try:
    sales_df = pd.read_csv('/content/drive/My Drive/TechHub_Sales_Data.csv')
    customers_df = pd.read_csv('/content/drive/My Drive/TechHub_Customers.csv')
    products_df = pd.read_csv('/content/drive/My Drive/TechHub_Products.csv')
    print("All three datasets loaded successfully.")
except FileNotFoundError:
    print("Error: One or more CSV files not found. Please ensure they are in '/content/drive/My Drive/'.")

    raise FileNotFoundError("Required CSV files not found. Please check file paths.")
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    raise Exception(f"Error loading data: {e}")


combined_df = pd.merge(sales_df, customers_df, on='customer_id', how='left')
print("Sales data merged with Customer data.")


combined_df = pd.merge(combined_df, products_df, on='product_id', how='left')
print("Combined data merged with Product data.")


combined_df['order_date'] = pd.to_datetime(combined_df['order_date'], errors='coerce')
combined_df['signup_date'] = pd.to_datetime(combined_df['signup_date'], errors='coerce')
combined_df['launch_date'] = pd.to_datetime(combined_df['launch_date'], errors='coerce')
print("Date columns converted to datetime format.")


print("\nChecking for missing values before final handling:")
print(combined_df.isnull().sum()[combined_df.isnull().sum() > 0])


numerical_cols = ['revenue', 'quantity', 'cost_price', 'list_price']
for col in numerical_cols:
    if col in combined_df.columns and combined_df[col].isnull().any():
        median_val = combined_df[col].median()
        combined_df[col].fillna(median_val, inplace=True)
        print(f"Filled missing values in '{col}' with median: {median_val}")


categorical_cols = ['age_group', 'gender', 'city', 'customer_type', 'loyalty_tier',
                    'product_category', 'supplier', 'customer_acquisition_channel', 'product_name']
for col in categorical_cols:
    if col in combined_df.columns and combined_df[col].isnull().any():
        combined_df[col].fillna('Unknown', inplace=True)
        print(f"Filled missing values in '{col}' with 'Unknown'")


initial_rows = len(combined_df)
combined_df.dropna(subset=['order_date', 'customer_id', 'product_id'], inplace=True)
rows_dropped = initial_rows - len(combined_df)
if rows_dropped > 0:
    print(f"Dropped {rows_dropped} rows due to missing essential IDs or order dates after merge.")

print("\nMissing values after handling:")
print(combined_df.isnull().sum()[combined_df.isnull().sum() > 0])

print("Missing values handled.")


combined_df['Profit Amount'] = combined_df['revenue'] - (combined_df['cost_price'] * combined_df['quantity'])
print("Calculated 'Profit Amount'.")


combined_df['Profit Margin %'] = (combined_df['Profit Amount'] / combined_df['revenue'].replace(0, pd.NA)) * 100

combined_df['Profit Margin %'].fillna(0, inplace=True)
print("Calculated 'Profit Margin %'.")


today = pd.Timestamp.now().normalize()
combined_df['Customer Tenure Days'] = (today - combined_df['signup_date']).dt.days

combined_df['Customer Tenure Days'] = combined_df['Customer Tenure Days'].apply(lambda x: max(0, x) if pd.notna(x) else x)
print("Calculated 'Customer Tenure Days'.")



clv_per_customer = combined_df.groupby('customer_id')['revenue'].sum().reset_index()
clv_per_customer.rename(columns={'revenue': 'Customer Lifetime Value'}, inplace=True)
combined_df = pd.merge(combined_df, clv_per_customer, on='customer_id', how='left')
print("Calculated 'Customer Lifetime Value'.")



combined_df['Product Age Days'] = (combined_df['order_date'] - combined_df['launch_date']).dt.days

combined_df['Product Age Days'] = combined_df['Product Age Days'].apply(lambda x: max(0, x) if pd.notna(x) else x)
print("Calculated 'Product Age Days'.")



output_file_path = '/content/drive/My Drive/TechHub_Combined_Data.csv'
combined_df.to_csv(output_file_path, index=False)
print(f"\n--- All data processed and saved to: {output_file_path} ---")

print("\nFirst 5 rows of the combined DataFrame:")
print(combined_df.head())

print("\nInfo of the combined DataFrame:")
combined_df.info()

Mounted at /content/drive
--- Starting Comprehensive Data Preparation for Tableau ---
All three datasets loaded successfully.
Sales data merged with Customer data.
Combined data merged with Product data.
Date columns converted to datetime format.

Checking for missing values before final handling:
Series([], dtype: int64)

Missing values after handling:
Series([], dtype: int64)
Missing values handled.
Calculated 'Profit Amount'.
Calculated 'Profit Margin %'.
Calculated 'Customer Tenure Days'.
Calculated 'Customer Lifetime Value'.
Calculated 'Product Age Days'.


/tmp/ipython-input-1-2472665722.py:76: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Profit Margin %'].fillna(0, inplace=True)



--- All data processed and saved to: /content/drive/My Drive/TechHub_Combined_Data.csv ---

First 5 rows of the combined DataFrame:
  order_date customer_id product_id product_category_x  revenue  quantity  \
0 2023-12-05       C0291      P0084           Monitors  3086.76         2   
1 2023-08-08       C2209      P0080            Tablets  1033.86         2   
2 2024-04-29       C1668      P0049            Tablets   644.56         2   
3 2024-03-17       C1452      P0099         Networking   659.20         4   
4 2023-02-25       C2759      P0095        Smartphones   174.85         1   

          region customer_acquisition_channel signup_date age_group  ...  \
0  West Midlands                     Referral  2020-10-13     56-65  ...   
1     South West                      Organic  2021-02-09     36-45  ...   
2     South East                      Organic  2022-09-22     36-45  ...   
3     North East                         Paid  2022-02-02     18-25  ...   
4         London        